<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1">Imports</a></span></li><li><span><a href="#electrostatic" data-toc-modified-id="electrostatic-2">electrostatic</a></span><ul class="toc-item"><li><span><a href="#Calculate" data-toc-modified-id="Calculate-2.1">Calculate</a></span></li><li><span><a href="#Pickle-dump" data-toc-modified-id="Pickle-dump-2.2">Pickle dump</a></span></li></ul></li></ul></div>

# Imports

In [1]:
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from importlib import reload
from common import baseclasses
reload(baseclasses); AWA=baseclasses.AWA
from TipEigenfields import ExpansionApproximation as EA
from common import plotting
import pickle
reload(EA); reload(EA.tip); reload(EA.az)

<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Bi2Se3_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "PMMA_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "sio2_300nm_extracted_epsilon_cone_A=2a.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_230K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "TaS2_eps_30K.csv"...


/Users/leo/anaconda3/lib/python3.6/site-packages/common/baseclasses.py:473: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  try: result=ndarray_operator(self,*args,**kwargs)
/Users/leo/anaconda3/lib/python3.6/site-packages/common/baseclasses.py:482: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  for axis_grid in axis_grids]


<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Erik_BSTS_epsilon.pickle"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_295K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "VO2_360K.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Stewart_V2O3film_insulating.csv"...
<material_types.TabulatedMaterialFromFile.__init__>:
	Loading tabulated material data from file "Stewart_V2O3film_metallic.csv"...
<plotting>:
	Registered colormaps "Warm" and "Warm_r"...
<plotting>:
	Registered colormaps "vanheum" and "vanheum_r"...
<plotting>:
	Registered colormaps "NT-MDT" and "NT-MDT_r"...
<plotting>:
	Registered colormaps "rainbow" and "rainbow_r"...
<plotting>:
	Registered colormaps "jingdi" and "jingdi_r"...
<plotting>:
	Registered colormaps "Halcyon" and "Halcyon_r"...
<plotting>:
	R

<module 'NearFieldOptics.PolarizationModels.azimuthal_charge' from '/Users/leo/anaconda3/lib/python3.6/site-packages/NearFieldOptics/PolarizationModels/azimuthal_charge.py'>

# electrostatic

## Calculate

In [2]:
#geometry and features of the probe
a=1
geometry='sphere' #options are 'hyperboloid', 'cone', 'ellipsoid','sphere'
L=2*a
load_freq=0 #in (cm-1), basically a small number, for quasi-electrostatic version of eigenfields
skin_depth=0

#settings for the eigenbasis calculation
Nqs=244
load_Nqs=244
load_Nzs=244
ztips = np.logspace(-1,1,12)*a    # range of tip-sample distance
N=5     #number of eigenbasis
Ezs=[];Vs=[]; Ers=[]; Ps=[]; Ds=[]; Rs=[]

# empty array for appending later
for n in np.arange(N)+1:
    Ezs.append([])
    Vs.append([])
    Ers.append([])
    Ps.append([])
    Ds.append([])
    Rs.append([])

for ztip in ztips:
    reload_eigenbasis=True #First time, we need to actually compute the eigenbasis
    for n in np.arange(N)+1:
        include_sample=False
        rmax_fields=12*a
        zmax_fields=a/100
        xrange=rmax_fields
        yrange=2*xrange+ztip

        #Iterate over eigenvalue index (can in principle go to infinity) and plot each eigenfield
        d=EA.EvaluateEigenfield(n=n,a=a,ztip=ztip,Nzs=2,Nrs=100,\
                                 zmin=zmax_fields/2,zmax=zmax_fields,rmax=rmax_fields,\
                                 from_sample=include_sample,beta=None,normalize=True,
                                 load_freq=load_freq,L=L,geometry=geometry,skin_depth=skin_depth,\
                                 reload_eigenbasis=reload_eigenbasis,Nqs=Nqs,load_Nqs=load_Nqs,load_Nzs=load_Nzs)
        reload_eigenbasis=False #We don't need to recompute eigenbasis in next rounds
        
        #save results
        Ezs[n-1].append(d['Ez'])
        Vs[n-1].append(d['potential'])
        Ers[n-1].append(d['Er'])
        Ps[n-1].append(d['eigenbasis']['Ps'][:N])
        Rs[n-1].append(d['eigenbasis']['Rs'][:N])

#convert items in dictionary to np array
for n in np.arange(N):
    Ezs[n] = np.array(Ezs[n])
    Vs[n] = np.array(Vs[n])
    Ers[n] = np.array(Ers[n])
    Ps[n] = np.array(Ps[n])
    Rs[n] = np.array(Rs[n])

#keep the spatial axes information
rs,zs=d['Ez'].axes
axis_names=d['Ez'].axis_names    

#convert list into AWA
n_max = len(Ezs)
eigen_axis = np.linspace(0,n_max-1,n_max)
axis_names = ['eigen #','tip-sample distance (a)','radial distance (a)']
P_AWA = AWA(Ps[0].T,axes=[eigen_axis,ztips],axis_names=axis_names[:2])
R_AWA = AWA(Rs[0].T,axes=[eigen_axis,ztips],axis_names=axis_names[:2])
Ez_AWA = AWA(np.array(Ezs)[:,:,:,0],axes=[eigen_axis,ztips,rs],axis_names=axis_names)
Er_AWA = AWA(np.array(Ers)[:,:,:,0],axes=[eigen_axis,ztips,rs],axis_names=axis_names)
V_AWA = AWA(np.array(Vs)[:,:,:,0],axes=[eigen_axis,ztips,rs],axis_names=axis_names)

/Users/leo/anaconda3/lib/python3.6/site-packages/TipEigenfields/ExpansionApproximation.py:74: RuntimeWarning: divide by zero encountered in true_divide
  Rs=-Es*Ls/alphas**2
/Users/leo/anaconda3/lib/python3.6/site-packages/TipEigenfields/ExpansionApproximation.py:74: RuntimeWarning: overflow encountered in true_divide
  Rs=-Es*Ls/alphas**2


## Pickle dump

In [3]:
result_dict={'P':P_AWA,'R':R_AWA,'V':V_AWA,'Ez':Ez_AWA,'Er':Er_AWA}

In [4]:
with open(geometry+'_load_freq='+str(load_freq)+'.pickle', 'wb') as handle:
    pickle.dump(result_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)